In [1]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib qt
import numpy as np
import cv2 as cv
import os
from typing import List
import matplotlib.pyplot as plt

In [2]:
def custom_pyramid(img, scales: List[float]) -> list:
    img_copy = img.copy()

    resized_imgs = list()

    # generamos nuestra "piramide" de tamaños arbitrarios
    for scale in scales:
        resized_imgs.append(cv.resize(img_copy, None, fx=scale, fy=scale))
        
    return resized_imgs

In [3]:
# Ruta de la carpeta con las imágenes
directorio = "./images"

# Lista para guardar las imágenes en gris
imagenes_gris = []

# Recorremos los archivos del directorio
for archivo in os.listdir(directorio):
    ruta_completa = os.path.join(directorio, archivo)

    # Verificamos que sea un archivo y termine en una extensión de imagen
    if os.path.isfile(ruta_completa) and archivo.lower().endswith(('.png', '.jpg')):
        img = cv.imread(ruta_completa, cv.IMREAD_GRAYSCALE)
        if img is not None:
            imagenes_gris.append(img)
        else:
            print(f"No se pudo leer la imagen: {archivo}")

print(f"Se leyeron {len(imagenes_gris)} imágenes en blanco y negro")

Se leyeron 7 imágenes en blanco y negro


In [4]:
img_aux = []
for img in imagenes_gris:
    img_aux.extend(custom_pyramid(img, np.linspace(0.2, 2, 10)))
imagenes_gris = img_aux

In [5]:
template = cv.imread('./template/pattern.png',0)

In [6]:
template_list = custom_pyramid(template, np.linspace(0.2, 2, 10))

In [7]:
len(template_list)

10

In [14]:
def match_template(img, template):
    img_copy = img.copy()
    template_copy = template.copy()
    w, h = template.shape[::-1]

    # --- Desenfoque Gaussiano ---
    img_blur = cv.GaussianBlur(img_copy, (5,5), 0)
    template_blur = cv.GaussianBlur(template_copy, (5,5), 0)

    # --- Bordes con Canny ---
    img_edges = cv.Canny(img_blur, 50, 150)
    template_edges = cv.Canny(template_blur, 50, 150)


    print(f"Buscando plantilla de tamaño {template_edges.shape[0]}x{template_edges.shape[1]} en imagen de tamaño {img_edges.shape[0]}x{img_edges.shape[1]}")
    res = cv.matchTemplate(img_edges, template_edges, cv.TM_CCOEFF_NORMED)
    # Fijamos un umbral de "coincidencia"

    threshold = 0.75
    flag = False
    loc = np.where( res >= threshold)
    for pt in zip(*loc[::-1]):
        cv.rectangle(img_copy, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        flag = True
    
    if flag:
        return img_copy
    else:
        return False

In [15]:
def search_template(img, template_list):
    img_copy = img.copy()

    result = []
    for i, template in enumerate(template_list):
        try:
            img_copy = match_template(img_copy, template)
            if img_copy:
                result.append(img_copy)
            # cv.imshow(f'Template Level {i}', img_copy)
            # cv.waitKey(0)
        except Exception as e:
            print(f"Error processing template level {i}: {e}")
            continue
    return result

In [16]:
def run_all_img(img_list, template_list):
    result = []
    for img in img_list:
        result.extend(search_template(img, template_list))
    return result

In [17]:
r = run_all_img(imagenes_gris, template_list)

Buscando plantilla de tamaño 35x80 en imagen de tamaño 278x278
Error processing template level 1: 'bool' object has no attribute 'copy'
Error processing template level 2: 'bool' object has no attribute 'copy'
Error processing template level 3: 'bool' object has no attribute 'copy'
Error processing template level 4: 'bool' object has no attribute 'copy'
Error processing template level 5: 'bool' object has no attribute 'copy'
Error processing template level 6: 'bool' object has no attribute 'copy'
Error processing template level 7: 'bool' object has no attribute 'copy'
Error processing template level 8: 'bool' object has no attribute 'copy'
Error processing template level 9: 'bool' object has no attribute 'copy'
Buscando plantilla de tamaño 35x80 en imagen de tamaño 556x556
Error processing template level 1: 'bool' object has no attribute 'copy'
Error processing template level 2: 'bool' object has no attribute 'copy'
Error processing template level 3: 'bool' object has no attribute 'copy

In [13]:
cv.imshow(f'Template Level', r[1])